In [67]:
from utils.experimentor import *
from utils.constants import *
import torch
import time
from torch_geometric.utils import to_dense_adj



In [68]:
config = {
    "dataset_name": Dataset.OGBN_ARXIV,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 10000,
    'num_of_runs': 10000,
    'patience_period': 1,
    
    #'should_visualize': False,
    'batch_size': 256,
    'test_batch_size': 256,
    'num_workers': 0,
    'force_cpu': False,
    'test_frequency': 5,
    'console_log_freq': 1,
    'do_train_tqdm_logging': True,
    
    'lr': 0.01,
    'num_of_layers': 3, 
    'num_heads': 1,
    'hidden_size': 256,
    'dropout': 0.25,  
    "use_layer_norm": True,
    
    'nbor_degree': 2,
    'adj_mode': AdjacencyMode.OneStep
}

In [75]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print(edge_index)

def one_step_sparse(edge_index, x):
    edge_weight = None
    if x>1: 
        edge_weight = torch.ones(edge_index.size(1))
        size = torch.Size([edge_index.size(1), edge_index.size(1)] + list(edge_weight.size())[1:])
        adj = torch.sparse.IntTensor(edge_index, edge_weight, size = size)
        
        #adj = to_dense_adj(edge_index)[0]
        for k in range(2, x+1):
            adj_k = 1/k * (torch.sparse.mm(adj, adj) - adj)
            
            #adj_k._values = (adj_k._values()>0)*1
            #adj = adj+adj_k

        #print(adj.to_dense())
        return adj
            
    return edge_index

avg_passed = 0
n= 0
for i in range(1000):   
    start = time.time_ns()
    hop2 = one_step_sparse(edge_index, 2)
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print(avg_passed/n)

print(hop2)

tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 3, 4, 4, 5]])
997308.8050314465
tensor(indices=tensor([[0, 0, 1, 1, 2, 2],
                       [1, 2, 3, 4, 4, 5]]),
       values=tensor([1., 1., 1., 1., 1., 1.]),
       size=(6, 6), nnz=6, layout=torch.sparse_coo)


In [70]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print(edge_index)

def one_step(edge_index, x):
    edge_weight = None
    if x>1: 
        edge_weight = torch.ones(edge_index.size(1))
        adj = to_dense_adj(edge_index)[0]
        for k in range(2,x+1):
            adj_k = (torch.linalg.matrix_power(adj, k)>0)*1 - adj
            
            edge_index_k = (adj_k > 0).nonzero().t()
            edge_weight_k = 1/k * torch.ones(edge_index_k.size(1))
            edge_index = torch.cat((edge_index, edge_index_k), dim=1)
            edge_weight = torch.cat((edge_weight, edge_weight_k), dim=0)

    return edge_index, edge_weight

avg_passed = 0
n= 0
for i in range(1000):   
    start = time.time_ns()
    hop2 = one_step(edge_index, 2)
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print(avg_passed/n)

print(hop2)

tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 3, 4, 4, 5]])
1000937.676056338
(tensor([[0, 0, 1, 1, 2, 2, 0, 0, 0],
        [1, 2, 3, 4, 4, 5, 3, 4, 5]]), tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.5000, 0.5000, 0.5000]))


In [71]:
# experimentor = Experimentor(config)
# experimentor.run()